# Import Starting Data

In [1]:
import pandas as pd #1
import os           #3

In [2]:
given_data_folder = 'data'

file_names = os.listdir(given_data_folder)
file_names.sort()
file_names

['MRegularSeasonDetailedResults.csv', 'bracket-2022.csv']

In [3]:
file_name = 'MRegularSeasonDetailedResults.csv'

In [4]:
file_path = given_data_folder + '/' + file_name 
given_df  = pd.read_csv(file_path)
print(len(given_df))

given_df.head(5)

100423


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


# Testing With The Network

In [5]:
import sys
sys.path.append("..")

In [6]:
import random
import time
import threading
import math

from copy import deepcopy

In [7]:
from network_classes.neuron      import Neuron
from network_classes.agent       import Agent
from network_classes.population  import Population

from network_classes.data_prep     import *
from network_classes.testingreport import TestingReport


from extra.progress_package import *

# Get each teams stats from a given season

### 2021 season

In [8]:
game_solutions_2021, team_dictionary_2021, headers_2021 = stats_for_season(given_df, 2021, print_report=False)

### 2022 season

In [9]:
game_solutions_2022, team_dictionary_2022, headers_2022 = stats_for_season(given_df, 2022, print_report=True)

data size: 
-----------
number of games for 2022 season:  3736 
number of columns in base data:   34 
number of rows:                   3736 


column titles and indicis: 
---------------------------
  0: Season      1: DayNum      2: WTeamID     3: WScore      4: LTeamID     5: LScore      6: WLoc      
  7: NumOT       8: WFGM        9: WFGA       10: WFGM3      11: WFGA3      12: WFTM       13: WFTA      
 14: WOR        15: WDR        16: WAst       17: WTO        18: WStl       19: WBlk       20: WPF       
 21: LFGM       22: LFGA       23: LFGM3      24: LFGA3      25: LFTM       26: LFTA       27: LOR       
 28: LDR        29: LAst       30: LTO        31: LStl       32: LBlk       33: LPF       


base stats report: 
-------------------
game_solutions length:    3736 games
team_dictionary length:    358 teams

team_dictionary[team_id] = [# of games played, [stats FOR totals], [stats AGAINST totals]]


example row:    
------------- 

  number of games played by team: 23 

   

## base stats with FOR and AGAINST

### 2021

In [10]:
# get team dictionary using custom stats
# ---------------------------------------
FA_team_dictionary_2021, FA_headers_list_2021 = for_and_against(team_dictionary_2021)


# change team dictionary to averages instead of totals
# -----------------------------------------------------
FA_team_averages_2021 = team_totals_to_averages(FA_team_dictionary_2021, FA_headers_list_2021, print_report=False)


# prepare the final input and solution data
# ------------------------------------------
FA_inputs_2021, FA_solutions_2021 = prep_inputs_and_solutions(FA_team_averages_2021, game_solutions_2021)


# set up final inputs and solutions
# ----------------------------------
inputs_2021    = FA_inputs_2021
solutions_2021 = FA_solutions_2021

### 2022

In [11]:
# get team dictionary using custom stats
# ---------------------------------------
FA_team_dictionary_2022, FA_headers_list_2022 = for_and_against(team_dictionary_2022)


# change team dictionary to averages instead of totals
# -----------------------------------------------------
FA_team_averages_2022 = team_totals_to_averages(FA_team_dictionary_2022, FA_headers_list_2022)


# prepare the final input and solution data
# ------------------------------------------
FA_inputs_2022, FA_solutions_2022 = prep_inputs_and_solutions(FA_team_averages_2022, game_solutions_2022)


# set up final inputs and solutions
# ----------------------------------
inputs_2022    = FA_inputs_2022
solutions_2022 = FA_solutions_2022

# Random Tests

In [12]:
import numpy as np

In [13]:
list_1 = [1.1,2.2,3.3]
list_2 = np.asarray(list_1)

list_3 = np.multiply(list_2, 0.8)

print(list_1)
print(list_2)
print(list_3)


[1.1, 2.2, 3.3]
[1.1 2.2 3.3]
[0.88 1.76 2.64]


In [14]:
#3220*7472
# 10*161*7472 = 12_029_920
# 10*322*7472 = 24_059_840

## Testing with splitting the input data into 10 parts

In [15]:
''' 
split_count = 10

inputs_splits = []
solutions_splits = []

for i in range(split_count):
    actual_length = len(inputs) // 2
    split_length  = (actual_length // split_count) * 2
    
    start_index =  i    * split_length
    end_index   = (i+1) * split_length
    
    inputs_splits.append(    inputs   [start_index:end_index].copy() )
    solutions_splits.append( solutions[start_index:end_index].copy() )
    
    print(len( inputs[start_index:end_index] ))
    
TEST_inputs = inputs[:2615].copy()
TEST_solutions = solutions[:2615].copy()
''';

# First Test (uses threading)

### variables for stat tracking

### initialize population

In [16]:
TEST_population = Population( len(inputs_2022[0]), 16)

for i in range(10):
    for agent in TEST_population.agents:
        agent.mutate(delete_neurons=True)

In [17]:
report_data = [inputs_2021, solutions_2021,
               inputs_2022, solutions_2022]

#report_data = [inputs_2021[:100], solutions_2021[:100], inputs_2022[:100], solutions_2022[:100]]

current_test = TestingReport(TEST_population)

### Training Loop

In [18]:
train_hours = 3
train_minutes = train_hours*60
train_seconds = train_minutes*60

print(train_hours, train_minutes, train_seconds) #started 5:00 am

3 180 10800


In [31]:
testing_time = 600 # (in seconds)

current_test.run_test(testing_time, report_data)

current_test.print_report()
print()
current_test.scoring_time_report()
print()
current_test.print_top_agent()

[##################################################] - 100.0%                                       


            current    prev       change  
            --------   --------   --------
timer        793.52s    191.18s    602.34s
steps            41          9         32 
steps/sec      0.05       0.05       0.05 
secs/step     18.82s     19.39s     18.82s
evolu time     0.93s      0.22s      0.71s
score time   792.53s    190.90s    601.63s
            --------   --------   --------
conf score      650        453        196 
base score     2153       2109         44 
accuracy      57.63%     56.45%      1.18%

		
id - [ c r, nc r, nc w,  c w] | ovrl | score |  conf | weight  (650)
--------------------------------------------------------------
 0 - [1794,  356,  376, 1210] | 2150 |   650 |   914 |   -265 
 1 - [ 794, 1082, 1074,  786] | 1876 |  -238 |    22 |   -260 
 2 - [1484,  610,  528, 1114] | 2094 |   415 |   687 |   -272 
 3 - [ 107, 1782, 1742,  105] | 1889 |  -227 |    42 |  

In [32]:
current_test.print_report()



            current    prev       change  
            --------   --------   --------
timer        793.52s    191.18s    602.34s
steps            41          9         32 
steps/sec      0.05       0.05       0.05 
secs/step     18.82s     19.39s     18.82s
evolu time     0.93s      0.22s      0.71s
score time   792.53s    190.90s    601.63s
            --------   --------   --------
conf score      650        453        196 
base score     2153       2109         44 
accuracy      57.63%     56.45%      1.18%

		
id - [ c r, nc r, nc w,  c w] | ovrl | score |  conf | weight  (650)
--------------------------------------------------------------
 0 - [1794,  356,  376, 1210] | 2150 |   650 |   914 |   -265 
 1 - [ 794, 1082, 1074,  786] | 1876 |  -238 |    22 |   -260 
 2 - [1484,  610,  528, 1114] | 2094 |   415 |   687 |   -272 
 3 - [ 107, 1782, 1742,  105] | 1889 |  -227 |    42 |   -269 
 4 - [1580,  485,  422, 1249] | 2065 |   346 |   605 |   -260 
 5 - [   0, 2057, 1679,    0] |

In [33]:
TEST_population.num_calculations

508

In [34]:
current_test.size_report(all_attributes=False)

41 steps

                        kB          neurons     avg size
                        --------    --------    --------
layer[0]                   11.66          26        0.45 
layer[1]                    7.45          18        0.41 
layer[2]                    7.17          18         0.4 
layer[3]                    4.65          14        0.33 
layer[4]                    0.25           1        0.25 


                        kB          mB
                        --------    --------
total_size                  31.5        0.03

